In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json
import pandas as pd
import users

cid = "4acff433ba4c4b1e97a5cacdb6358f52"
secret = "bcd9bc8e6123468085c44fa911a613ad"
for name in users.user_list:
    username = name
    print(username)
    
client_credentials_manager = SpotifyClientCredentials(cid, secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
username = "klinga88"


In [7]:
file = pd.read_csv("music.csv",encoding='latin1', skiprows=1)
file.head()

,Position,Track Name,Artist,Streams,URL
0,1,I Love It (& Lil Pump),Kanye West,4989832,https://open.spotify.com/track/4S8d14HvHb70Imc...
1,2,In My Feelings,Drake,3842246,https://open.spotify.com/track/2G7V7zsVDxg1yRs...
2,3,Lucky You (feat. Joyner Lucas),Eminem,3673656,https://open.spotify.com/track/60SdxE8apGAxMiR...
3,4,Promises (with Sam Smith),Calvin Harris,3491483,https://open.spotify.com/track/5N5k9nd479b1xpD...
4,5,Eastside (with Halsey & Khalid),benny blanco,3452605,https://open.spotify.com/track/0d2iYfpKoM0QCKv...


In [ ]:
playlists = sp.user_playlists(username)
my_playlists = {}
while playlists:
    for i, playlist in enumerate(playlists['items']):
        playlist_owner = playlist["uri"].split(":")[2]
        playlist_name = playlist['name']
        playlist_id = playlist["uri"].split(":")[4]
        if playlist_owner == username:
            my_playlists[playlist_name] = playlist_id
    if playlists['next']:
        playlists = sp.next(playlists)
    else:
        playlists = None
        
print(json.dumps(my_playlists, indent=4))

In [ ]:
#get tracks in each playlist
songs = []
sp_playlist = sp.user_playlist_tracks(username, playlist_id="0qRjzddROXVAy1sxiOfwcr")
tracks = sp_playlist['items']
print(json.dumps(tracks, indent=4))

In [ ]:


print(f"length of playlist: {len(tracks)}")
#get each song, song URI, artist, artist URI, album, album URI, album img url
for i in range(len(tracks)):
    try:
        song_name = tracks[i]["track"]["name"]
        song_uri = tracks[i]["track"]["uri"]
        artist_name = tracks[i]["track"]["artists"][0]["name"]# @TODO Update to get all artists on track not just first 
        artist_uri = tracks[i]["track"]["artists"][0]["uri"]
        album_name = tracks[i]["track"]["album"]["name"]
        album_uri = tracks[i]["track"]["album"]["uri"]
        album_img = tracks[i]["track"]["album"]["images"][0]["url"]
        popularity = tracks[i]["track"]["popularity"]
        songs.append([song_name,song_uri,artist_name,artist_uri,album_name,album_uri,album_img,popularity])
    except Exception as e:
        print(f"Exception on track number {i}, {e}")
print(json.dumps(songs,indent=4))

In [ ]:
#create dataframe
df_tracks = pd.DataFrame(songs, columns=["song_name", "song_uri", "artist_name","artist_uri","album_name","album_uri","album_img","album_popularity"])
df_tracks.groupby(["artist_name"])["album_popularity"].mean().sort_values()

In [ ]:
track_ids = df_tracks["song_uri"]
track_features = []
for track in track_ids:
    track = track.split(":")[2]
    track_features.append(sp.audio_features(track))
    
print(json.dumps(track_features, indent=4))

In [ ]:
features = pd.DataFrame(track_features, columns=["danceability"])
features.head()
#@TODO Add radar overlaying each song from a playlist